### Package Loading

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from google.colab import drive, auth
from sklearn.dummy import DummyRegressor
from google.colab import drive
from google.colab import auth
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Mount Google Drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')

### Cleaning Dataset

In [ ]:


master_df = pd.read_csv('IWC_Work_Orders_Extract.csv')

# GROUP 2: Nick S., Nick G.
master_df2 = master_df.copy()


columns_to_remove = ['ORDER_ID', 'FUNCTIONAL_AREA_NODE_1_MODIFIED',
                    'FUNCTIONAL_AREA_NODE_2_MODIFIED', 'FUNCTIONAL_AREA_NODE_3_MODIFIED',
                    'FUNCTIONAL_AREA_NODE_4_MODIFIED', 'FUNCTIONAL_AREA_NODE_5_MODIFIED',
                     'EQUIP_VALID_TO', 'PRODUCTION_LOCATION']

master_df2 = master_df2.drop(columns=columns_to_remove, axis=1)


columns_to_fill = ['MAINTENANCE_PLAN', 'MAINTENANCE_ITEM', 'FUNCTIONAL_LOC', 'EQUIPMENT_DESC', 'EQUIP_CAT_DESC',
                  'EQUIP_START_UP_DATE', 'EQUIP_VALID_FROM', 'EQUIPMENT_ID']
master_df2[columns_to_fill] = master_df[columns_to_fill].fillna('Unknown')

planned_df = master_df2[master_df2['MAINTENANCE_ACTIVITY_TYPE'] == 'Planned']


columns_to_drop = ['EXECUTION_START_DATE', 'EXECUTION_FINISH_DATE',
                   'ACTUAL_START_TIME', 'ACTUAL_FINISH_TIME', 'EQUIP_VALID_FROM',
                   'EQUIP_START_UP_DATE', 'MAINTENANCE_PLAN', 'MAINTENANCE_ITEM', 'ACTUAL_WORK_IN_MINUTES']
planned_df.drop(columns=columns_to_drop, inplace=True)



### Planned Association Rule Mining

In [ ]:
# Remove rows with 'Unknown' in any column
planned_df_no_unknown = planned_df[~planned_df.apply(lambda row: row.astype(str).str.contains('Unknown')).any(axis=1)]

# Create transactions for association rule mining, ensuring all elements are strings
transactions = planned_df_no_unknown[['FUNCTIONAL_LOC', 'ORDER_DESCRIPTION', 'MAINTENANCE_ACTIVITY_TYPE']].astype(str).values.tolist()

# Encode transactions and apply apriori
te = TransactionEncoder()
df = pd.DataFrame(te.fit_transform(transactions), columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)

# Generate and display rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
print(rules)

The above rules show the relationship between ORDER_DESCRIPTION, MAINTENANCE_PLAN, and FUNCTIONAL_LOC. We can see how the different values for each attribute show up in relation to one another and we can estimate their relationships based on that. For example, the two highest lift rules are 000000022943 -> E-STOP FOR ELEC MONTHLY SHUTDOWN and its inverse. The confidence of the latter is 100% indicating that whenever there is an emergency stop for monthly shutdown that specified maintenence plan is always used. However if we look at the first rule it does not have 100% confidence indicating that it also has other uses. Other things we can observe in these rules are which production locations are linked to which rules or maintenence plans.There are also rules that contain combinations of the other attributes as both antecedents and consequents indicating strong relationships between for example a certain order desc and maintenence plan with a specific factory location.